<a href="https://colab.research.google.com/github/MarcelaMonteiroMontenegroGallo/Python/blob/master/Kinesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load a dataframe From File 

In [ ]:

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark

     |████████████████████████████████| 212.4 MB 58 kB/s 
     |████████████████████████████████| 198 kB 35.9 MB/s 


In [ ]:
import io


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark.sql.types import * 

**Definir o schema**

In [ ]:
emp_schema = StructType([
StructField('firstname', StringType()),   
StructField('lastname', StringType()),
StructField('personal_email', StringType()),             
StructField('ssn', StringType()),
StructField('office', StringType()),
StructField('title', StringType()),
StructField('gender', StringType()),
StructField('org', StringType()),
StructField('accrued_holidays', IntegerType()),
StructField('salary', IntegerType()),
StructField('bonus', IntegerType()),
StructField('event_time', TimestampType())
])

**Ler o fluxo do Kinesis Streams**

In [ ]:
kinesisDf = spark\
  .readStream \
  .format("kinesis")\
  .option("streamName", "Kinesis-Spark-Streaming-Analytics")\
  .option("initialPosition","earliest")\
  .option("region", "us-east-1") \
  .option("awsAccesskey","") 
  .option("awsSecretKey","")
  .load()        
            )


**Extraia dados e aplique o esquema**

In [ ]:
from pyspark.sql.functions import * 

In [ ]:
empdf =kinesisDF \
  .selectExpr("cast(data as STRING) jsonData") \
  .select(from_json("jsonData",emp_schema).alias("emp")) \
  .select("emp.*")\

**Definir função da janela**

In [ ]:
empcountsdf = (
  empdf
    .groupBy(
      empdf.org,
      window(empdf.event_time,"1 minute"))
    .count()  
    )

**Iniciar Spark Stream**




In [ ]:
spark.conf.set("spark.sql.shuffle.partitions". "2")

query =(
    empcountsdf
      .writeStream
      .format("memory")
      .queryname("counts")
      .outputMode("complete")
      .start()
)

**Resultados da Consulta**

In [ ]:
spark.sql('select * fromn counts order by desc').show()